## Amazon Kindle Book Review for Sentiment Analysis
### Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

### Content

- 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.

### Columns/Features

- asin           - ID of the product, like B000FA64PK
- helpful        - helpfulness rating of the review - example: 2/3.
- overall        - rating of the product.
- reviewText     - text of the review (heading).
- reviewTime     - time of the review (raw).
- reviewerID     - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName   - name of the reviewer.
- summary        - summary of the review (description).
- unixReviewTime - unix timestamp.

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import nltk
from bs4 import BeautifulSoup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


invalid command name "136637749483456set"
    while executing
"136637749483456set"
    ("after" script)
invalid command name "136637749482560set"
    while executing
"136637749482560set"
    ("after" script)
invalid command name "136636905530816set"
    while executing
"136636905530816set"
    ("after" script)
invalid command name "136636905529088set"
    while executing
"136636905529088set"
    ("after" script)
invalid command name "136636905532928set"
    while executing
"136636905532928set"
    ("after" script)
invalid command name "136637749482752set"
    while executing
"136637749482752set"
    ("after" script)
invalid command name "136636905534400set"
    while executing
"136636905534400set"
    ("after" script)
invalid command name "136637749480192set"
    while executing
"136637749480192set"
    ("after" script)
invalid command name "136636905535424set"
    while executing
"136636905535424set"
    ("after" script)
invalid command name "136636905536576set"
    while executing
"1

True

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df=pd.read_csv("all_kindle_review .csv")

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


#### Aim of the project is to sentiment Analysis, so we need the text and rating only.

In [ ]:
df=df[['reviewText',"rating"]]

In [ ]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
df.shape

(12000, 2)

- Total 12,000 reviews in the dataset.

In [ ]:
df.describe()

,rating
count,12000.000000
mean,3.250000
std,1.421619
min,1.000000
25%,2.000000
50%,3.500000
75%,4.250000
max,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  12000 non-null  object
 1   rating      12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [ ]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

#### There are 5 different rating the dataset, we convert the 5 values into 2 category (positive-1, negative-0).

In [ ]:
df1=df[df['rating']!=3]

In [ ]:
df1.head()

,reviewText,rating
1,Great short read. I didn't want to put it dow...,5
4,I did not expect this type of book to be in li...,4
5,Aislinn is a little girl with big dreams. Afte...,5
6,This has the makings of a good story... unfort...,2
7,I got this because I like collaborated short s...,4


In [ ]:
df1['rating'].value_counts()

rating
5    3000
4    3000
2    2000
1    2000
Name: count, dtype: int64

In [ ]:
df1['rating']=df1['rating'].apply(lambda x: 0 if x<3 else 1)

In [ ]:
df1['rating'].value_counts()

rating
1    6000
0    4000
Name: count, dtype: int64

In [ ]:
df1.head()

,reviewText,rating
1,Great short read. I didn't want to put it dow...,1
4,I did not expect this type of book to be in li...,1
5,Aislinn is a little girl with big dreams. Afte...,1
6,This has the makings of a good story... unfort...,0
7,I got this because I like collaborated short s...,1


In [ ]:
df['reviewText'].value_counts()

reviewText
Jace Rankin may be short, but he's nothing to mess with, as the man who was just hauled out of the saloon by the undertaker knows now. He's a famous bounty hunter in Oregon in the 1890s who, when he shot the man in the saloon, just finished a years long quest to avenge his sister's murder and is now trying to figure out what to do next. When the snotty-nosed farm boy he just rescued from a gang of bullies offers him money to kill a man who forced him off his ranch, he reluctantly agrees to bring the man to justice, but not to kill him outright. But, first he needs to tell his sister's widower the news.Kyla "Kyle" Springer Bailey has been riding the trails and sleeping on the ground for the past month while trying to find Jace. She wants revenge on the man who killed her husband and took her ranch, amongst other crimes, and she's not so keen on the detour Jace wants to take. But she realizes she's out of options, so she hides behind her boy persona as best she can and tries t

### Preprocessing the Text data
- Remove duplicates
- Remove missing values
- Remove Stopwords
- Remove non alphabatic
- Stemming or Lemmatization

In [ ]:
df1.duplicated().sum()

0

- No duplicates

In [ ]:
df1.isnull().sum()

reviewText    0
rating        0
dtype: int64

- No missing values

In [ ]:
df1['reviewText'].iloc[0]

"Great short read.  I didn't want to put it down so I read it all in one sitting.  The sex scenes were great between the two males and one female character...a bit surprising - I never thought you could do that!  I learned something new and really enjoyed reading this book!  This is a great way to get all hot and bothered and take advantage of your significant other(s)!"

In [ ]:
#convert all the character into lower case
df1['reviewText']=df1['reviewText'].str.lower()

In [ ]:
df1['reviewText'].iloc[0]

"great short read.  i didn't want to put it down so i read it all in one sitting.  the sex scenes were great between the two males and one female character...a bit surprising - i never thought you could do that!  i learned something new and really enjoyed reading this book!  this is a great way to get all hot and bothered and take advantage of your significant other(s)!"

In [ ]:
# remove the characters other than alphabets and numericals
df1['reviewText']=df1['reviewText'].apply(lambda x:re.sub("[^A-Z a-z 0-9]+"," ",x))

In [ ]:
df1['reviewText'].iloc[0]

'great short read   i didn t want to put it down so i read it all in one sitting   the sex scenes were great between the two males and one female character a bit surprising   i never thought you could do that   i learned something new and really enjoyed reading this book   this is a great way to get all hot and bothered and take advantage of your significant other s '

In [ ]:
# remove the stopwords
df1['reviewText']=df1['reviewText'].apply(lambda x:" ".join([y for y in x.split() if not y in stopwords.words("english")]))

In [ ]:
df1['reviewText'].iloc[0]

'great short read want put read one sitting sex scenes great two males one female character bit surprising never thought could learned something new really enjoyed reading book great way get hot bothered take advantage significant'

In [ ]:
# remove the hyperlinks and website linkes

df1['reviewText']=df1['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://[\w_-]+(?:\.[\w_-]+)+[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-]?', '', x))

In [ ]:
df1['reviewText'].iloc[0]

'great short read want put read one sitting sex scenes great two males one female character bit surprising never thought could learned something new really enjoyed reading book great way get hot bothered take advantage significant'

In [ ]:
## remove the html tags if the text contains
df1['reviewText'] = df1['reviewText'].apply(lambda x: re.sub(r'<.*?>', '', x))

In [ ]:
df1['reviewText'].iloc[0]

'great short read want put read one sitting sex scenes great two males one female character bit surprising never thought could learned something new really enjoyed reading book great way get hot bothered take advantage significant'

In [ ]:
df1['reviewText'] = df1['reviewText'].apply(lambda x: " ".join(x.split()))

In [ ]:
df1['reviewText'].iloc[0]

'great short read want put read one sitting sex scenes great two males one female character bit surprising never thought could learned something new really enjoyed reading book great way get hot bothered take advantage significant'

### Text Before the Preprocessing

In [ ]:
df['reviewText'].iloc[0]

'Jace Rankin may be short, but he\'s nothing to mess with, as the man who was just hauled out of the saloon by the undertaker knows now. He\'s a famous bounty hunter in Oregon in the 1890s who, when he shot the man in the saloon, just finished a years long quest to avenge his sister\'s murder and is now trying to figure out what to do next. When the snotty-nosed farm boy he just rescued from a gang of bullies offers him money to kill a man who forced him off his ranch, he reluctantly agrees to bring the man to justice, but not to kill him outright. But, first he needs to tell his sister\'s widower the news.Kyla "Kyle" Springer Bailey has been riding the trails and sleeping on the ground for the past month while trying to find Jace. She wants revenge on the man who killed her husband and took her ranch, amongst other crimes, and she\'s not so keen on the detour Jace wants to take. But she realizes she\'s out of options, so she hides behind her boy persona as best she can and tries to ke

In [ ]:
def preprocess(text):
    # convert the all characters into lower case
    text=text.lower()
    # remove the character other than alphabets and numericals
    text=re.sub("[^A-Z a-z 0-9]+"," ",text)
    #remove the stop words
    text=" ".join([y for y in text.split() if not y in stopwords.words("english")])
    #remove the hyperlinks
    text=re.sub(r'(http|https|ftp|ssh)://[\w_-]+(?:\.[\w_-]+)+[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-]?', '',text)
    # remove the html tags
    text=re.sub(r'<.*?>', '', text)
    return text

In [ ]:
preprocess(df['reviewText'].iloc[0])

'jace rankin may short nothing mess man hauled saloon undertaker knows famous bounty hunter oregon 1890s shot man saloon finished years long quest avenge sister murder trying figure next snotty nosed farm boy rescued gang bullies offers money kill man forced ranch reluctantly agrees bring man justice kill outright first needs tell sister widower news kyla kyle springer bailey riding trails sleeping ground past month trying find jace wants revenge man killed husband took ranch amongst crimes keen detour jace wants take realizes options hides behind boy persona best tries keep pace confrontation along way gets shot jace discovers kyle kyla come clean whole reason needs scoundrel dead hope still help book share touching moments slow blooming romance kyla find good reason fear men hide behind boy persona watching jace slowly pull shell help conquer fears endearing pain real deeply rooted disappear face sexiness neither understandable aversion marriage magically disappear round nookie would

In [ ]:
df1['reviewText']=df1['reviewText'].apply(lambda x: " ".join([WordNetLemmatizer().lemmatize(y) for y in x.split()]))

In [ ]:
df1['reviewText'].iloc[0]

'great short read want put read one sitting sex scene great two male one female character bit surprising never thought could learned something new really enjoyed reading book great way get hot bothered take advantage significant'

In [ ]:
df2=df1

In [ ]:
df2.head()

,reviewText,rating
1,great short read want put read one sitting sex...,1
4,expect type book library pleased find price right,1
5,aislinn little girl big dream death older brot...,1
6,making good story unfortunately disappoints te...,0
7,got like collaborated short story alot time tw...,1


In [ ]:
df2.to_csv("cleaned_kindle_reivews.csv",index=False)

In [ ]:
df2=pd.read_csv("cleaned_kindle_reivews.csv")

### Splitting the data into the training and testing data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df2['reviewText'],df2['rating'],test_size=0.2,random_state=42)

In [ ]:
print("X train shape",X_train.shape)
print("y train shape",y_train.shape)
print("x test shape",X_test.shape)
print("y test shape",y_test.shape)


X train shape (8000,)
y train shape (8000,)
x test shape (2000,)
y test shape (2000,)


### Text to vector

### Bag of Words(BOW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

In [ ]:
vecotrizer=CountVectorizer()
x_train_bow=vecotrizer.fit_transform(X_train).toarray()
x_test_bow=vecotrizer.transform(X_test).toarray()

In [ ]:
x_train_bow.shape

(8000, 22485)

In [ ]:
x_train_bow[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
print("X train BOW shape",x_train_bow.shape)

print("x test BOW shape",x_test_bow.shape)


X train BOW shape (8000, 22485)
x test BOW shape (2000, 22485)


### TF-IDF

In [ ]:
tfidf=TfidfVectorizer()
x_train_tfidf=tfidf.fit_transform(X_train).toarray()
x_test_tfidf=tfidf.transform(X_test).toarray()

In [ ]:
print("X train TFIDF shape",x_train_tfidf.shape)

print("x test TFIDF shape",x_test_tfidf.shape)


X train TFIDF shape (8000, 22485)
x test TFIDF shape (2000, 22485)


### Word2Vec

In [ ]:
# tokenize the sentence
X_train_tokens=[review.split() for review in X_train]
X_test_tokens=[review.split() for review in X_test]

In [ ]:
X_train_tokens[0]

['read',
 'writing',
 'collection',
 'thoroughly',
 'enjoying',
 'reading',
 'look',
 'forward',
 'one',
 'previously',
 'read']

In [ ]:
word2vec=Word2Vec(sentences=X_train_tokens,vector_size=100,window=5,min_count=1,workers=4)

In [ ]:
# build Average word2vec model
def avgword2vec(tokens,model):
    avg_vec=[]
    for review in tokens:
        vectors=[model.wv[word] for word in review if word in model.wv]
        if vectors:
            avg_vec.append(sum(vectors)/len(vectors))
        else:
            avg_vec.append([0] * model.vector_size)
    return avg_vec


In [ ]:
x_train_w2v=avgword2vec(X_train_tokens,word2vec)

In [ ]:
len(x_train_w2v)

8000

In [ ]:
x_test_w2v=avgword2vec(X_test_tokens,word2vec)

In [ ]:
len(x_test_w2v)

2000

### Model Training and Evalution

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
models={"Logistic Regression":LogisticRegression(),
       "Decision Tree":DecisionTreeClassifier(),
       "RandomForestClassifier":RandomForestClassifier(),
       "SVM":SVC()
       }

#### Train the model with Bag of words

In [ ]:
for key,value in models.items():
    print(key)
    value.fit(x_train_bow,y_train)
    y_pred=value.predict(x_test_bow)
    print(accuracy_score(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

Logistic Regression
0.8855
[[ 682  129]
 [ 100 1089]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       811
           1       0.89      0.92      0.90      1189

    accuracy                           0.89      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.89      0.89      0.89      2000

Decision Tree
0.7095
[[509 302]
 [279 910]]
              precision    recall  f1-score   support

           0       0.65      0.63      0.64       811
           1       0.75      0.77      0.76      1189

    accuracy                           0.71      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.71      0.71      0.71      2000

RandomForestClassifier
0.846
[[ 580  231]
 [  77 1112]]
              precision    recall  f1-score   support

           0       0.88      0.72      0.79       811
           1       0.83      0.94      0.88      1189

    accuracy                

#### Train the Model with TF-IDF

In [ ]:
for key,value in models.items():
    print(key)
    print("*"*20)
    value.fit(x_train_tfidf,y_train)
    y_pred=value.predict(x_test_tfidf)
    print(accuracy_score(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

Logistic Regression
********************
0.888
[[ 656  155]
 [  69 1120]]
              precision    recall  f1-score   support

           0       0.90      0.81      0.85       811
           1       0.88      0.94      0.91      1189

    accuracy                           0.89      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.89      0.89      0.89      2000

Decision Tree
********************
0.7205
[[527 284]
 [275 914]]
              precision    recall  f1-score   support

           0       0.66      0.65      0.65       811
           1       0.76      0.77      0.77      1189

    accuracy                           0.72      2000
   macro avg       0.71      0.71      0.71      2000
weighted avg       0.72      0.72      0.72      2000

RandomForestClassifier
********************
0.849
[[ 577  234]
 [  68 1121]]
              precision    recall  f1-score   support

           0       0.89      0.71      0.79       811
           1       0.

#### Train the Model with Word2vec

In [ ]:
for key,value in models.items():
    print(key)
    print("-"*50)
    value.fit(x_train_w2v,y_train)
    y_pred=value.predict(x_test_w2v)
    print(accuracy_score(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

Logistic Regression
--------------------------------------------------
0.7755
[[553 258]
 [191 998]]
              precision    recall  f1-score   support

           0       0.74      0.68      0.71       811
           1       0.79      0.84      0.82      1189

    accuracy                           0.78      2000
   macro avg       0.77      0.76      0.76      2000
weighted avg       0.77      0.78      0.77      2000

Decision Tree
--------------------------------------------------
0.665
[[471 340]
 [330 859]]
              precision    recall  f1-score   support

           0       0.59      0.58      0.58       811
           1       0.72      0.72      0.72      1189

    accuracy                           0.67      2000
   macro avg       0.65      0.65      0.65      2000
weighted avg       0.66      0.67      0.66      2000

RandomForestClassifier
--------------------------------------------------
0.757
[[545 266]
 [220 969]]
              precision    recall  f1-score   su

#### Svm with TF-IDF vectorizer perfoms well than other models with Bag of words and Wor2vec.

In [ ]:
svm=SVC()
svm.fit(x_train_tfidf,y_train)

SVC()

In [ ]:
logistic=LogisticRegression()
logistic.fit(x_train_tfidf,y_train)
joblib.dump(logistic,"logistic_model.pkl")

['logistic_model.pkl']

In [ ]:
import joblib
import pickle


# Save Bag of Words vectorizer
joblib.dump(vecotrizer, 'vectorizer_bow.pkl')

# Save TF-IDF vectorizer
joblib.dump(tfidf, 'vectorizer_tfidf.pkl')

# Save Word2Vec vectorizer (if applicable)
with open('vectorizer_word2vec.pkl', 'wb') as f:
    pickle.dump(word2vec, f)

# Save the trained SVM model
joblib.dump(svm, 'svm_model.pkl')

print("Models and vectorizers saved successfully!")


Models and vectorizers saved successfully!


### Conclusion
- The dataset contains 12,000 datapoints.
- Reduce the datapoints by convert the rating (0 to 5) into "Positive" review and "Negative" Review by removing the rating 3.
- Below 3 rating consider as "Negative" and Above 3 rating consider as "Positive".
- All the Review text are preprocessed and Convert them into vectors.
- Convert the text with Bag of words, TF-IDF and Word2Vec.
- TF-IDF vectorizer performs well with SVM model with accuracy 89%.